In [1]:
import pandas as pd

In [2]:
class SegmentsDataProcessor:
    
    def __init__(self, filename):
        self.data = pd.read_csv(filename, sep='\t', comment='#')
        self.data = self.process_data(self.data)
    
    def process_data(self, data):
        data = self.reshape_data(data)
        data = self.process_values(data)
        data = self.rename_chromosome_column(data)
        data = self.cast_column_types(data)
        data = self.remove_Y_rows(data)
        
        return data
        
        
    def get_cnv_segments(self, sex='female'):    
        cnv_data = self.data.copy()
        cnv_data = self.remove_not_cnv_rows(cnv_data)
        cnv_data = self.rename_event_column(cnv_data)
        cnv_data = self.transform_cn_values(cnv_data)
        cnv_data = self.cast_cn(cnv_data)
        cnv_data.reset_index(inplace=True, drop=True)
        
        return cnv_data
    
    
    def get_ai_segments(self):    
        ai_data = self.data.copy()
        ai_data = self.remove_not_ai_rows(ai_data)
        ai_data = self.drop_event_column(ai_data)
        ai_data.reset_index(inplace=True, drop=True)
        
        return ai_data
    
    
    def get_loh_segments(self):
        loh_data = self.data.copy()
        loh_data = self.remove_not_loh_rows(loh_data)
        loh_data = self.drop_event_column(loh_data)
        #loh_data.reset_index(inplace=True, drop=True)
        loh_data = pd.concat([self.get_ai_segments(), loh_data]).reset_index(drop=True)
        
        return loh_data


    def reshape_data(self, data):
        data = data.loc[:, 'Chromosome Region':'Length']

        data.loc[:,'Start'] = pd.Series('', index=data.index)
        data.loc[:,'End'] = pd.Series('', index=data.index)

        return data


    def process_values(self, data):
        data[['Chromosome Region', 'Start', 'End']] = data['Chromosome Region'].apply(self.process_chromosome_reg_col)
        
        data.loc[:,'Chromosome Region'] = data.loc[:,'Chromosome Region'].apply(lambda x: x.strip('chr'))
        data.loc[:,'Start'] = data.loc[:,'Start'].apply(lambda x: x.replace(',', ''))
        data.loc[:,'End'] = data.loc[:,'End'].apply(lambda x: x.replace(',', ''))
        data.loc[:,'Length'] = data.loc[:,'Length'].apply(lambda x: x - 1)

        return data


    def process_chromosome_reg_col(self, text):
        chromosome, rest = text.split(':')
        start, end = rest.split('-')
        
        return pd.Series([chromosome, start, end])


    def rename_chromosome_column(self, data):
        data.rename(columns = {'Chromosome Region':'Chromosome'}, inplace = True)

        return data


    def remove_Y_rows(self, data):
        data = data.loc[data['Chromosome'] != 'Y']

        return data


    def cast_column_types(self, data):
        data.loc[:,'Start'] = data.loc[:,'Start'].astype(str).astype('int64')
        data.loc[:,'End'] = data.loc[:,'End'].astype(str).astype('int64')

        return data

    
    def transform_cn_values(self, data):
        category_to_number = {'Copy Number' : {'Big Loss' : 0,
                                'CN Loss' : 1,
                                'CN Gain' : 3,
                                'High Copy Gain' : 4}
                         }
        data.replace(category_to_number, inplace=True)
        
        return data
    
    
    def remove_not_cnv_rows(self, data):
        data = data.loc[(data['Event'] != 'LOH') & (data['Event'] != 'Allelic Imbalance') & (data['Event'] != 'Homozygous Copy Loss')]

        return data
    
    
    def rename_event_column(self, data):
        data.rename(columns = {'Event':'Copy Number'}, inplace = True)

        return data
    
    
    def cast_cn(self, data):
        data.loc[:,'Copy Number'] = data.loc[:,'Copy Number'].astype(str).astype('int64')

        return data
    
    
    def remove_not_ai_rows(self, data):
        data = data.loc[data['Event'] == 'Allelic Imbalance']

        return data
    
    
    def remove_not_loh_rows(self, data):
        data = data.loc[data['Event'] == 'LOH']

        return data
    
    
    def drop_event_column(self, data):
        data.drop(columns='Event', inplace=True)
        
        return data

In [3]:
pd.set_option('display.max_rows', 1000)

In [4]:
sdp = SegmentsDataProcessor('../datasets/CytoScan.Segmental.Reports/OS001_events.txt')

sdp.data

,Chromosome,Event,Length,Start,End
0,1,CN Gain,1961263,3699709,5660972
1,1,CN Gain,1898487,6923109,8821596
2,1,LOH,1455266,12392108,13847374
3,1,CN Gain,3223207,12392108,15615315
4,1,LOH,514454,13954989,14469443
5,1,Allelic Imbalance,966717,14648598,15615315
6,1,Allelic Imbalance,509988,18705597,19215585
7,1,LOH,659327,19215585,19874912
8,1,LOH,1119125,22683593,23802718
9,1,LOH,1531696,26781719,28313415


In [5]:
cnv_data = sdp.get_cnv_segments()

cnv_data

,Chromosome,Copy Number,Length,Start,End
0,1,3,1961263,3699709,5660972
1,1,3,1898487,6923109,8821596
2,1,3,3223207,12392108,15615315
3,1,3,1522221,60470133,61992354
4,1,3,13965170,69521787,83486957
5,1,3,2073749,96008174,98081923
6,1,3,1220630,98085037,99305667
7,1,3,1743373,103266261,105009634
8,1,1,37760,152750381,152788141
9,1,1,86690164,162560457,249250621


In [6]:
ai_data = sdp.get_ai_segments()

ai_data

,Chromosome,Length,Start,End
0,1,966717,14648598,15615315
1,1,509988,18705597,19215585
2,1,13502876,31997927,45500803
3,1,1630216,47597845,49228061
4,1,9240572,52160657,61401229
5,1,2022427,61807957,63830384
6,1,4470528,65502270,69972798
7,1,2027600,70488402,72516002
8,1,1886927,74182018,76068945
9,1,3177942,76557496,79735438


In [7]:
loh_data = sdp.get_loh_segments()

loh_data

,Chromosome,Length,Start,End
0,1,966717,14648598,15615315
1,1,509988,18705597,19215585
2,1,13502876,31997927,45500803
3,1,1630216,47597845,49228061
4,1,9240572,52160657,61401229
5,1,2022427,61807957,63830384
6,1,4470528,65502270,69972798
7,1,2027600,70488402,72516002
8,1,1886927,74182018,76068945
9,1,3177942,76557496,79735438


In [10]:
sdp2 = SegmentsDataProcessor2('../datasets/p6.Inform/I062.033.WGS.Tumor_events.txt', 'P6.Rec7')

sdp2.data

,Chromosome,Event,Length,Start,End
0,1,CN Loss,157435,104172098,104329533
1,1,CN Loss,2534626,145295296,147829922
2,1,CN Loss,2068346,149849729,151918075
3,1,CN Gain,1352303,216006715,217359018
4,1,CN Gain,565566,222719682,223285248
5,1,CN Gain,5443247,224232394,229675641
6,1,CN Gain,3867195,235821067,239688262
7,1,CN Gain,4300807,244949814,249250621
8,2,CN Gain,22335482,10805837,33141319
9,2,Homozygous Copy Loss,7615,33141319,33148934


In [11]:
cnv_data2 = sdp2.get_cnv_segments()

cnv_data2

,Chromosome,Copy Number,Length,Start,End
0,1,1,157435,104172098,104329533
1,1,1,2534626,145295296,147829922
2,1,1,2068346,149849729,151918075
3,1,3,1352303,216006715,217359018
4,1,3,565566,222719682,223285248
5,1,3,5443247,224232394,229675641
6,1,3,3867195,235821067,239688262
7,1,3,4300807,244949814,249250621
8,2,3,22335482,10805837,33141319
9,2,3,593337,33148934,33742271


In [12]:
ai_data2 = sdp2.get_ai_segments()

ai_data2

,Chromosome,Length,Start,End
0,6,676671,58102296,58778967
1,9,75480428,65733003,141213431
2,16,452919,46386883,46839802
3,17,359046,21902620,22261666
4,21,358327,10822954,11181281


In [13]:
loh_data2 = sdp2.get_loh_segments()

loh_data2

,Chromosome,Length,Start,End
